In [34]:
import pandas as pd
import numpy as np

In [35]:
train = pd.read_csv('/home/huy/Desktop/HCMUS/image_classification/datasets/CIFAR/test.csv', index_col=0)

In [36]:
train['llm_idx'] = np.nan
train['llm_label'] = np.nan

In [37]:
answers = {
    "(A)": 'airplane',
    "(B)": 'automobile',
    "(C)": 'bird',
    "(D)": 'cat',
    "(E)": 'deer',
    "(F)": 'dog',
    "(G)": 'frog',
    "(H)": 'horse',
    "(I)": 'ship',
    "(J)": 'truck',
}

In [38]:
# extract LLM labels from description
def extract_llm_label(description, is_first=False):
    idx = {}
    if '(A)' in description:
        idx['(A)'] = description.index('(A)')
    if '(B)' in description:
        idx['(B)'] = description.index('(B)')
    if '(C)' in description:
        idx['(C)'] = description.index('(C)')
    if '(D)' in description:
        idx['(D)'] = description.index('(D)')  
    if '(E)' in description:
        idx['(E)'] = description.index('(E)')
    if '(F)' in description:
        idx['(F)'] = description.index('(F)')
    if '(G)' in description:
        idx['(G)'] = description.index('(G)')
    if '(H)' in description:
        idx['(H)'] = description.index('(H)')
    if '(I)' in description:
        idx['(I)'] = description.index('(I)')
    if '(J)' in description:
        idx['(J)'] = description.index('(J)')    
    
    if len(idx) == 2 and is_first:
        min_key = min(idx, key=idx.get)
        # Return a dictionary with the key and its minimum value
        return {min_key: idx[min_key]}

    return idx   

train['llm_idx'] = train['description'].apply(extract_llm_label)

In [39]:
# extract LLM label from description if len idx key is 1
def extract_llm_label_from_idx(idx):
    if len(idx) == 1:
        return answers[list(idx.keys())[0]]
    else:
        return np.nan

train['llm_label'] = train['llm_idx'].apply(extract_llm_label_from_idx)

# handle split string

In [40]:
error = train[train['llm_label'].isna()]

In [41]:
error['sub_description'] = error['description'].apply(lambda x: x.split('The main object in the image')[-1])

/tmp/ipykernel_1968904/3223177211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['sub_description'] = error['description'].apply(lambda x: x.split('The main object in the image')[-1])


In [42]:
error['llm_idx'] = error['sub_description'].apply(extract_llm_label, is_first=True)

/tmp/ipykernel_1968904/2769801696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_idx'] = error['sub_description'].apply(extract_llm_label, is_first=True)


In [43]:
error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)

/tmp/ipykernel_1968904/502944615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)


In [44]:
train['llm_label'] = train['llm_label'].fillna(error['llm_label'])

# get index by answer string

In [45]:
error = train[train['llm_label'].isna()]

In [46]:
# extract LLM labels from description
def extract_by_label(description, is_first=False):
    idx = {}
    if 'airplane' in description:
        idx['(A)'] = description.index('airplane')
    if 'automobile' in description:
        idx['(B)'] = description.index('automobile')
    if 'bird' in description:
        idx['(C)'] = description.index('bird')
    if 'cat' in description:
        idx['(D)'] = description.index('cat')  
    if 'deer' in description:
        idx['(E)'] = description.index('deer')
    if 'dog' in description:
        idx['(F)'] = description.index('dog')
    if 'frog' in description:
        idx['(G)'] = description.index('frog')
    if 'horse' in description:
        idx['(H)'] = description.index('horse')
    if 'ship' in description:
        idx['(I)'] = description.index('ship')
    if 'truck' in description:
        idx['(J)'] = description.index('truck')    
        
    if is_first:
        try:
            min_key = min(idx, key=idx.get)
            # Return a dictionary with the key and its minimum value
            return {min_key: idx[min_key]}
        except:
            return ''

    return idx   

error['llm_idx'] = error['description'].apply(extract_by_label, is_first=True)

/tmp/ipykernel_1968904/1638846304.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_idx'] = error['description'].apply(extract_by_label, is_first=True)


In [47]:
error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)

/tmp/ipykernel_1968904/502944615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)


In [48]:
train['llm_label'] = train['llm_label'].fillna(error['llm_label'])

# FINISH

In [49]:
train.llm_label.value_counts()

llm_label
horse         1818
truck         1218
cat           1027
automobile    1015
airplane       952
dog            946
ship           876
bird           864
frog           718
deer           529
Name: count, dtype: int64

In [50]:
sum(train['label'] == train['llm_label']) / len(train) # 0.999

0.7561

In [33]:
# save to csv
train.to_csv('tested.csv')